# 🤖 Chatbot Educativo con Naive Bayes y TF-IDF
Este notebook implementa un chatbot en español utilizando técnicas de PLN, Naive Bayes, y TF-IDF. Incluye limpieza de texto, corrección ortográfica, evaluación del modelo y un chatbot interactivo.

## 📦 Importación de librerías necesarias

In [2]:
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
from collections import Counter

## 📥 Descarga de recursos de NLTK

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\darly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\darly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\darly\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\darly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## ⚙️ Configuración de herramientas de lenguaje

In [52]:
#stopwords_es = set(stopwords.words('spanish'))
stopwords_es = set(stopwords.words('spanish')) - {"no", "cómo", "dónde", "cuándo"}

stemmer = SnowballStemmer("spanish")
spell = SpellChecker(language='es')

## 🗂️ Dataset
El dataset contiene frases clasificadas en 12 categorías.

In [165]:
dataset = [
    # SALUDO
    ("Hola", "saludo"), ("Buenos días", "saludo"), ("¿Qué tal?", "saludo"), ("¿Cómo estás?", "saludo"),
    ("Saludos", "saludo"), ("Buenas tardes", "saludo"), ("Buenas noches", "saludo"),
    ("¿Qué hay de nuevo?", "saludo"), ("¡Hola, qué gusto!", "saludo"), ("¿Cómo te va?", "saludo"),

    # QUEJA
    ("Estoy molesto por el servicio", "queja"), ("No me gusta el trato que recibí", "queja"),
    ("La atención fue muy lenta", "queja"), ("El producto llegó dañado", "queja"),
    ("No contestan mis llamadas", "queja"), ("No estoy conforme con la calidad", "queja"),
    ("Tuve un problema con la compra", "queja"), ("El envío se retrasó", "queja"),
    ("No funciona como esperaba", "queja"), ("El empaque estaba roto", "queja"),

    # RECLAMO
    ("Quiero hacer un reclamo formal", "reclamo"), ("Deseo que me reembolsen", "reclamo"),
    ("Voy a presentar una queja oficial", "reclamo"), ("Estoy haciendo un reclamo de garantía", "reclamo"),
    ("Reclamo por la mala atención recibida", "reclamo"), ("No recibí lo que compré", "reclamo"),
    ("Reclamo por doble cobro", "reclamo"), ("Quiero un cambio del producto", "reclamo"),
    ("No me llegó la factura", "reclamo"), ("El reclamo fue ignorado", "reclamo"),

    # PETICIÓN
    ("Necesito saber más sobre el servicio", "petición"), ("Quiero más información", "petición"),
    ("¿Tienen catálogo en línea?", "petición"), ("Envíenme más detalles por correo", "petición"),
    ("¿Dónde puedo ver los productos?", "petición"), ("¿Cómo puedo comprar?", "petición"),
    ("Solicito asesoría", "petición"), ("Me pueden orientar con el proceso", "petición"),
    ("¿Hay más opciones disponibles?", "petición"), ("¿Tienen promociones?", "petición"),

    # DESPEDIDA
    ("Hasta luego", "despedida"), ("Nos vemos", "despedida"), ("Gracias, adiós", "despedida"),
    ("Chao", "despedida"), ("Gracias por todo", "despedida"), ("Fue un gusto, adiós", "despedida"),
    ("Nos vemos pronto", "despedida"), ("Hasta la próxima", "despedida"), ("Cuídate", "despedida"),
    ("Buen día, adiós", "despedida"),

    # CONTACTO
    ("¿Dónde los puedo contactar?", "contacto"), ("¿Cuál es su número de atención?", "contacto"),
    ("¿Tienen WhatsApp?", "contacto"), ("¿Hay un correo para escribirles?", "contacto"),
    ("¿Tienen redes sociales?", "contacto"), ("¿Dónde puedo encontrarlos?", "contacto"),
    ("¿Atienden por teléfono?", "contacto"), ("¿Tienen línea de atención?", "contacto"),
    ("¿Dónde está la oficina?", "contacto"), ("¿Cómo puedo hablar con un asesor?", "contacto"),

    # HORARIO
    ("¿Cuál es el horario de atención?", "horario"), ("¿Trabajan los fines de semana?", "horario"),
    ("¿A qué hora abren?", "horario"), ("¿Hasta qué hora están disponibles?", "horario"),
    ("¿Tienen atención 24/7?", "horario"), ("¿Qué días trabajan?", "horario"),
    ("¿Están abiertos hoy?", "horario"), ("¿Puedo llamarlos en la noche?", "horario"),
    ("¿Tienen horario continuo?", "horario"), ("¿Hay atención los festivos?", "horario"),

    # PRECIO
    ("¿Cuánto cuesta el producto?", "precio"), ("¿Cuál es el precio del servicio?", "precio"),
    ("¿Tienen precios especiales?", "precio"), ("¿Hay descuentos?", "precio"),
    ("¿Cuánto vale el envío?", "precio"), ("¿Tienen ofertas?", "precio"),
    ("¿Es muy caro?", "precio"), ("¿Cuál es el precio por unidad?", "precio"),
    ("¿Puedo pagar a cuotas?", "precio"), ("¿Aceptan pagos con tarjeta?", "precio"),

    # DEVOLUCIÓN
    ("¿Puedo devolver el producto?", "devolución"), ("¿Cuál es la política de devoluciones?", "devolución"),
    ("¿Aceptan cambios?", "devolución"), ("¿Puedo cambiar por otro modelo?", "devolución"),
    ("¿Cuánto tiempo tengo para devolver?", "devolución"), ("¿Puedo pedir reembolso?", "devolución"),
    ("¿Me devuelven el dinero?", "devolución"), ("¿Dónde hago la devolución?", "devolución"),
    ("¿Tengo que pagar por devolver?", "devolución"), ("¿Cómo gestiono una devolución?", "devolución"),

    # SOPORTE
    ("Tengo un problema técnico", "soporte"), ("¿Cómo puedo solucionar un error?", "soporte"),
    ("¿Hay soporte en línea?", "soporte"), ("¿Tienen ayuda técnica?", "soporte"),
    ("Mi sistema no responde", "soporte"), ("No puedo entrar a la plataforma", "soporte"),
    ("Me aparece un error en pantalla", "soporte"), ("¿Dónde reporto un fallo?", "soporte"),
    ("Necesito asistencia técnica", "soporte"), ("Tengo un problema con la app", "soporte"),

    # AGRADECIMIENTO
    ("Muchas gracias", "agradecimiento"), ("Gracias por su ayuda", "agradecimiento"),
    ("Agradezco mucho su atención", "agradecimiento"), ("Muy amables, gracias", "agradecimiento"),
    ("Gracias por responder", "agradecimiento"), ("Qué buen servicio, gracias", "agradecimiento"),
    ("Gracias por su tiempo", "agradecimiento"), ("Estoy agradecido", "agradecimiento"),
    ("Gracias por todo", "agradecimiento"), ("Muy agradecido", "agradecimiento"),

    # FELICITACIÓN
    ("Excelente atención", "felicitación"), ("Muy buen servicio", "felicitación"),
    ("Estoy muy satisfecho", "felicitación"), ("Felicitaciones por su trabajo", "felicitación"),
    ("Gracias, todo salió perfecto", "felicitación"), ("Muy bien hecho", "felicitación"),
    ("Quiero felicitar al equipo", "felicitación"), ("Todo funcionó excelente", "felicitación"),
    ("Estoy feliz con el resultado", "felicitación"), ("Buen trabajo", "felicitación")
]  


In [140]:
len(dataset)

120

## 🧹 Función de limpieza de frases

In [145]:
def limpiar_frase(frase):
    frase = unicodedata.normalize("NFD", frase)
    frase = frase.encode("ascii", "ignore").decode("utf-8")
    frase = frase.lower()
    tokens = word_tokenize(frase, language='spanish')
   # tokens_corregidos = [spell.correction(palabra) for palabra in tokens if palabra is not None]
  #  tokens_corregidos = [spell.correction(palabra) or palabra for palabra in tokens]
    palabras_clave = {"como", "estas", "hola", "gracias", "no", "si"}
    tokens_corregidos = [
        palabra if palabra in palabras_clave else spell.correction(palabra) or palabra for palabra in tokens ]

    palabras_limpias = [
    stemmer.stem(p) for p in tokens_corregidos
    if p and p.isalpha() and (p not in stopwords_es or p in palabras_clave) and len(p) > 2
]

   
    print(" ".join(palabras_limpias))
    return " ".join(palabras_limpias)

In [152]:
def limpiar_frase(frase):
    # Normaliza caracteres Unicode y convierte a minúsculas
    frase = unicodedata.normalize("NFD", frase)
    frase = frase.encode("ascii", "ignore").decode("utf-8")
    frase = frase.lower()

    # Tokeniza en español
    tokens = word_tokenize(frase, language='spanish')

    # Palabras que no deben ser corregidas (claves para la intención)
    palabras_clave = {"como", "estas", "hola", "gracias", "no", "si"}

    # Corrige solo si no es palabra clave. Si la corrección da None, conserva la original
    tokens_corregidos = [
        palabra if palabra in palabras_clave else spell.correction(palabra) or palabra
        for palabra in tokens
    ]

    # Filtrado + stemmin
    palabras_limpias = [
       stemmer.stem(p) for p in tokens #va cooregido
        if p and p.isalpha() and (p not in stopwords_es) and len(p) > 2
    ]
    print(" ".join(palabras_limpias))

    return " ".join(palabras_limpias)


In [167]:
def limpiar_frase(frase):
    # Normaliza caracteres Unicode y convierte a minúsculas
    frase = unicodedata.normalize("NFD", frase)
    frase = frase.encode("ascii", "ignore").decode("utf-8")
    frase = frase.lower()

    # Tokeniza en español
    tokens = word_tokenize(frase, language='spanish')

    # Palabras que no deben ser corregidas (claves para la intención)
    palabras_clave = {"como", "estas", "hola", "gracias", "no", "si"}

    # Corrige solo si no es palabra clave. Si la corrección da None, conserva la original
    tokens_corregidos = [
        palabra if palabra in palabras_clave else spell.correction(palabra) or palabra
        for palabra in tokens
    ]

    # Filtrado + stemmin
   
    palabras_limpias = [
        stemmer.stem(p) for p in tokens
        if p and p.strip() != "" and p.isalpha() and (p not in stopwords_es) and len(p) > 2
]

    print(" ".join(palabras_limpias))

    return " ".join(palabras_limpias)

## 📊 Preparación de datos y revisión del balance de clases

In [168]:
frases = [limpiar_frase(texto) for texto, etiqueta in dataset]
etiquetas = [etiqueta for texto, etiqueta in dataset]
conteo_clases = Counter(etiquetas)
for etiqueta, cantidad in conteo_clases.items():
    print(f"{etiqueta}: {cantidad} frases")

hol
buen dias
tal

salud
buen tard
buen noch
nuev
hol gust

molest servici
gust trat recibi
atencion lent
product lleg dan
contest llam
conform calid
problem compr
envi retras
funcion esper
empaqu rot
quier hac reclam formal
dese reembols
voy present quej oficial
hac reclam garanti
reclam mal atencion recib
recibi compr
reclam dobl cobr
quier cambi product
lleg factur
reclam ignor
necesit sab mas servici
quier mas inform
catalog line
envienm mas detall corre
pued ver product
pued compr
solicit asesori
pued orient proces
mas opcion dispon
promocion
lueg
vem
graci adi
cha
graci
gust adi
vem pront
proxim
cuidat
buen dia adi
pued contact
numer atencion
whatsapp
corre escrib
red social
pued encontr
atiend telefon
line atencion
oficin
pued habl asesor
horari atencion
trabaj fin seman
hor abren
hor estan dispon
atencion
dias trabaj
estan abiert hoy
pued llam noch
horari continu
atencion festiv
cuant cuest product
preci servici
preci especial
descuent
cuant val envi
ofert
car
preci unid
pued p

## ✂️ División train/test y vectorización con TF-IDF

In [169]:
X_train, X_test, y_train, y_test = train_test_split(frases, etiquetas, test_size=0.2, stratify=etiquetas, random_state=42)
#vectorizador = TfidfVectorizer() #sin n-gramas (de a uno)
vectorizador = TfidfVectorizer(ngram_range=(1, 2), min_df=1) #con bigramas

X_train_vec = vectorizador.fit_transform(X_train)
X_test_vec = vectorizador.transform(X_test)

## 📈 Entrenamiento del modelo y evaluación

In [170]:
modelo = MultinomialNB()
modelo.fit(X_train_vec, y_train)
y_pred = modelo.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9166666666666666
                precision    recall  f1-score   support

agradecimiento       0.67      1.00      0.80         6
      contacto       1.00      1.00      1.00         6
     despedida       1.00      0.83      0.91         6
    devolución       0.67      1.00      0.80         6
  felicitación       1.00      1.00      1.00         6
       horario       1.00      1.00      1.00         6
      petición       1.00      1.00      1.00         6
        precio       1.00      1.00      1.00         6
         queja       1.00      1.00      1.00         6
       reclamo       1.00      0.50      0.67         6
        saludo       1.00      0.67      0.80         6
       soporte       1.00      1.00      1.00         6

      accuracy                           0.92        72
     macro avg       0.94      0.92      0.91        72
  weighted avg       0.94      0.92      0.91        72



## 🤖 Clasificador interactivo

In [171]:
def clasificar_interactivo(frase):
    frase_limpia = limpiar_frase(frase)
    vector = vectorizador.transform([frase_limpia])
    print("____", modelo.predict(vector)[0])
    return modelo.predict(vector)[0]

## 💬 Diccionario de respuestas múltiples por categoría

In [172]:
respuestas = {
    "saludo": ["¡Hola! ¿Cómo puedo ayudarte hoy?", "¡Hola! Bienvenido, dime en qué te puedo ayudar.", "Saludos, ¿en qué te puedo colaborar?"],
    "queja": ["Lamentamos los inconvenientes.", "Entendemos tu malestar.", "Gracias por reportarlo."],
    "reclamo": ["Vamos a revisar tu reclamo.", "Tu reclamo ha sido registrado.", "Estamos gestionando tu reclamo."],
    "petición": ["Procesando tu solicitud.", "Te enviaremos información.", "Recibirás respuesta pronto."],
    "despedida": ["Gracias por contactarnos.", "Fue un gusto ayudarte.", "Chao, vuelve pronto."],
    "contacto": ["Puedes contactarnos al 123.", "Correo: contacto@empresa.com", "WhatsApp: 3210000000"],
    "horario": ["Horario: Lun-Vie 9-18", "También sábados.", "No atendemos festivos."],
    "precio": ["Precio base: $99.99", "Consulta promociones online.", "Mira el catálogo."],
    "devolución": ["30 días para devolver.", "Política flexible.", "Desde tu cuenta web."],
    "soporte": ["Centro de ayuda online.", "Describe el error.", "Estamos aquí para ayudarte."],
    "agradecimiento": ["¡Gracias por tu confianza!", "Nos alegra ayudarte.", "Siempre es un placer."],
    "felicitación": ["¡Nos alegra tu satisfacción!", "Gracias por tus palabras.", "Seguiremos mejorando."]
}

## 🧠 Chatbot en consola

In [173]:
def iniciar_chatbot():
    print("Chatbot Educativo (escribe 'salir' para terminar)")
    while True:
        entrada = input("Tú: ")
        if entrada.lower() == "salir":
            print("Chatbot: ¡Hasta luego!")
            break
        categoria = clasificar_interactivo(entrada)
        print(f"Chatbot: {random.choice(respuestas.get(categoria, ['Lo siento, no entendí tu solicitud.']))}")

In [174]:
iniciar_chatbot()

Chatbot Educativo (escribe 'salir' para terminar)


Tú:  hola


hol
____ saludo
Chatbot: ¡Hola! ¿Cómo puedo ayudarte hoy?


Tú:  quiero despejar una duda


quier despej dud
____ felicitación
Chatbot: Gracias por tus palabras.


...

peticion
____ agradecimiento
Chatbot: ¡Gracias por tu confianza!


Tú:  donde estan ubicados


estan ubic
____ horario
Chatbot: No atendemos festivos.


Tú:  salir


Chatbot: ¡Hasta luego!
